In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F          
from torch.utils.data import DataLoader  
from torch.utils.data import Subset      
from torchvision import datasets, transforms
from torchsummary import summary
import torchvision.models as models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
# Data loading
train_data_dir = r'C:\Users\computer\Desktop\chest\train'
test_data_dir = r'C:\Users\computer\Desktop\chest\test'

# Training transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adds color variation
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)), 
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# Testing transformations (no augmentation)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Load the training dataset with training transforms
train_dataset = datasets.ImageFolder(root=train_data_dir, transform=train_transform)
trainloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Load the testing dataset with testing transforms
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=test_transform)
testloader = DataLoader(test_dataset, batch_size=8, shuffle=False)


# Number of classes
num_classes = len(train_dataset.classes)
print (f'Number of class: {num_classes}')
class_names = train_dataset.classes
print (f'Name of class: {class_names}')
print (f'Train dataset:{len(train_dataset)}')
print (f'Test dataset:{len(test_dataset)}')

# Dataloaders dictionary
dataloaders = {
    'train': trainloader,
    'val': testloader  # Using testloader as validation set
}

Number of class: 2
Name of class: ['NORMAL', 'PNEUMONIA']
Train dataset:5216
Test dataset:624


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Freeze Layers Function
def freeze_layers(model, num):
    layers = list(model.children())
    for i, layer in enumerate(layers[:num]):
        for param in layer.parameters():
            param.requires_grad = False
    return model

In [5]:
# Evaluate Model Function
def evaluate_model(num_layers_to_freeze):
    # Load ResNet-18 and freeze layers
    model = models.resnet18(pretrained=True)
    model = freeze_layers(model, num_layers_to_freeze)
    model.fc = nn.Linear(model.fc.in_features, 10)  # CIFAR-10 has 10 classes
    model = model.to(device)  # Move model to GPU if available

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

    # Train for 2 epochs
    model.train()
    for epoch in range(2):
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Return validation accuracy as fitness
    return correct / total


In [6]:
import random
np.random.seed(42)
random.seed(42)

In [7]:
# PSO Implementation
def pso(num_particles=10, num_iterations=2):
    num_layers = 10  # Maximum number of layers to freeze in ResNet-18
    w = 0.5  # Inertia weight
    c1 = 1   # Cognitive coefficient
    c2 = 1   # Social coefficient

    # Initialize particles
    particles = np.random.randint(0, num_layers + 1, size=num_particles)
    velocities = np.random.uniform(-1, 1, size=num_particles)
    personal_best_positions = particles.copy()
    personal_best_scores = np.full(num_particles, -np.inf)
    global_best_position = particles[0]
    global_best_score = -np.inf

    for _ in range(num_iterations):
        for i in range(num_particles):
            # Evaluate fitness of the current particle
            fitness = evaluate_model(particles[i])
            print(f"Particle {i}, Layers to freeze: {particles[i]}, Fitness: {fitness}")

            # Update personal best
            if fitness > personal_best_scores[i]:
                personal_best_scores[i] = fitness
                personal_best_positions[i] = particles[i]

            # Update global best
            if fitness > global_best_score:
                global_best_score = fitness
                global_best_position = particles[i]

            # Update velocity and position
            r1, r2 = random.random(), random.random()
            velocities[i] = (
                w * velocities[i] +
                c1 * r1 * (personal_best_positions[i] - particles[i]) +
                c2 * r2 * (global_best_position - particles[i])
            )
            particles[i] = int(np.clip(particles[i] + velocities[i], 0, num_layers))

    print(f"Best number of layers to freeze: {global_best_position}, Best fitness: {global_best_score}")
    return global_best_position

# Run PSO
optimal_layers = pso()

C:\Users\computer\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\computer\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Particle 0, Layers to freeze: 6, Fitness: 0.9230769230769231
Particle 1, Layers to freeze: 3, Fitness: 0.8685897435897436
Particle 2, Layers to freeze: 10, Fitness: 0.9134615384615384
Particle 3, Layers to freeze: 7, Fitness: 0.9294871794871795
Particle 4, Layers to freeze: 4, Fitness: 0.8798076923076923
Particle 5, Layers to freeze: 6, Fitness: 0.9182692307692307
Particle 6, Layers to freeze: 9, Fitness: 0.9022435897435898
Particle 7, Layers to freeze: 2, Fitness: 0.9166666666666666
Particle 8, Layers to freeze: 6, Fitness: 0.9166666666666666
Particle 9, Layers to freeze: 10, Fitness: 0.8974358974358975


KeyboardInterrupt: 